In [ ]:
!pip install openai llama_index datasets ragas

In [ ]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key:··········


In [ ]:
# Dataset 로드
from datasets import load_dataset

ds = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split='train[:20]')
data = ds.to_pandas()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
docs = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [ ]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

# Generator_llm. : Golden Dataset 생성 시 활용하는 모델
generator_llm = OpenAI(model="gpt-4o-mini")
# Critic_llm : 각종 지표에 대한 계산 모델
critic_llm = OpenAI(model="gpt-4o")
embeddings = OpenAIEmbedding(model="text-embedding-3-small")

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# 생성할 테스트 쿼리 유형별 분포 정의
distributions = {
    simple:0.4,
    reasoning:0.2,
    multi_context:0.2,
    conditional:0.2
}
generator_naive = TestsetGenerator.from_llama_index(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings,
)

# 제너레이터 이용하여 독스로부터 샘플 쿼리 5개 생성
testset_naive = generator_naive.generate_with_llamaindex_docs(docs, 5, distributions)
testset_naive.to_pandas()

embedding nodes:   0%|          | 0/304 [00:00<?, ?it/s]

Generating:   0%|          | 0/5 [00:00<?, ?it/s]

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,공고 및 설명 절차가 인정되기 위해서는 어떤 조건들이 충족되어야 하는가?,"[�반수 동의가 인정되기 위해서는, ① 근로자들이 피고가 신설하려고 하는 ‘직급정년...",공고 및 설명 절차가 인정되기 위해서는 다음의 조건들이 충족되어야 한다: ① 근로자...,simple,[{}],True
1,What are Canada's official languages and what ...,[��? 짧은 대답은 둘 다입니다. 영어와 프랑스어는 모두 캐나다의 공식 언어이며 ...,Canada's official languages are English and Fr...,simple,[{}],True
2,캐나다 다문화가 교육에 미치는 영향은?,"[ 우수한 교육 프로그램, 경이로운 자연, 다문화 사회 및 저렴한 생활비를 제공합니...",The answer to given question is not present in...,reasoning,[{}],True
3,How do Canada's global traits affect students'...,[��도 있습니다. 대학에서 기숙사에 머물기를 기대합니까? 솔직히 어느 쪽도 다른 ...,The context does not provide specific informat...,multi_context,"[{}, {}]",True
4,이순신 장군의 교훈에 따른 해군기지 건설의 목적은?,"[�고 밝히고 있다. 또한, 19세기 말 일본 해군사관학교에서는 ""이순신전술전략""이...",The answer to given question is not present in...,conditional,[{}],True


In [ ]:
#한국어 패치가 필요!
generator = TestsetGenerator.from_llama_index(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=embeddings
)

language = "korean"

#추후 테스트셋 생성에 사용할 제너레이터 오브젝트 한국어에 adaptation 진행
generator.adapt(language, evolutions=[simple, reasoning, conditional, multi_context])
generator.save(evolutions=[simple, reasoning, multi_context, conditional])

In [ ]:
# 생성할 테스트 쿼리 유형별 분포 정의
distributions = {
    simple:0.4,
    reasoning:0.2,
    multi_context:0.2,
    conditional:0.2
}

# 한국어로 Adapt시킨 제너레이터 이용하여 독스로부터 샘플 쿼리 15개 생성
testset = generator.generate_with_llamaindex_docs(docs, 15, distributions, with_debugging_logs=True)
testset.to_pandas()

embedding nodes:   0%|          | 0/304 [00:00<?, ?it/s]

[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['캐나다의 교육 프로그램', '다문화 사회', '저렴한 생활비', '대학 선택', '유학']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['비상 예비 자금', '월급 6개월 치', 'CMA', 'MMF', '맞벌이 부부']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['법인회생', '엘케이비 기업법무 그룹', 'M&A', '사모펀드', '법률 서비스']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['노무현', '대선 홍보 현수막', '서거 이후 논쟁', '정치 의식', '독재정권 시대']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['국방부', '상급자의 비리와 부패', '하급자 성범죄', '성욕 원천차단', '유신시대 회귀']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['제주 롯데 아트빌라스', '세계적인 건축 거장', '제주 자연', '건축물', 'DA그룹']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['주인집 할머니', '모욕', '신자유주의', '한나 아렌트', '가나자와대학교 법학부 교수']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['제임스테이트블랙 기념상', '폴딩 스타', '맨부커상 후보', '빌화이트헤드상', 'LGBT 작가']}
[ragas.testset.extractor.DEBUG] topics: {'keyphrases': ['통치 구조', '인간의 감정과 이성', '전체주의 사회', '극단적 기독교', '저자와 주인공']}
[ragas.tes

Generating:   0%|          | 0/15 [00:00<?, ?it/s]

[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 3, 'relevance': 3, 'score': 2.75}
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 3, 'relevance': 3, 'score': 2.75}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['DK 토크관', '디저트 행사', '디자인과 기술', '청년 디자이너', 'DK 잡페어']
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['DK 토크관', '디저트 행사', '디자인과 기술', '청년 디자이너', 'DK 잡페어']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 1, 'relevance': 2, 'score': 1.5}
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 1, 'relevance': 2, 'score': 1.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['에리히 프롬', '현대인의 도덕적 책임', '자아 경험', '정신분석의 임상적 측면', '디스토피아 소설']
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['에리히 프롬', '현대인의 도덕적 책임', '자아 경험', '정신분석의 임상적 측면', '디스토피아 소설']
[ragas.testset.filters.DEBUG] context scoring:

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,노량해전의 결과와 일본 해군의 변화에 대해 설명해 주세요.,[��상자 10명을 제외하고는 피해를 본 것이 없었다. 그러나 일본은 전선 47척이...,"노량해전의 결과로 일본은 전선 47척이 격침되고 12척이 나포되었으며 9,000명이...",simple,[{}],True
1,"DK 잡페어에서 어떤 프로그램이 진행되었고, 어떤 기회가 제공되었나요?",[간을 가졌다. 뿐만 아니라 DK 토크관에서는 이슈를 만드는 사람과의 만남 디자인 ...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,simple,[{}],True
2,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점은 무엇인가요?,[대한민국 근대 도시사는 신도시 역사라 불리울 정도로 우리는 다양한 도시건설 노하우...,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점으로는 베트남에서의 신도시 사업...,simple,[{}],True
3,레고 부스트와 함께 아이들의 상상력과 창의력을 어떻게 키울 수 있나요?,"[�� 나오게 할 수도 있는데, 음성을 변조할 수도 있으니 아이들이 너무 좋아 하더...","레고 부스트와 함께 아이들은 직접 움직임을 줄 수 있어 성취감을 느끼고, 교감 능력...",simple,[{}],True
4,기업 승계가 효율적으로 진행된 사례는 어떤 상황에서 발생했나요?,[가가 부도 위기에 처해 IMF로부터 자금을 지원받아 국가 부도를 면한 외환위기 사...,기업 승계가 효율적으로 진행된 사례는 2008년 국제적인 금융위기와 같은 경제 위기...,simple,[{}],True
5,색상/거리 센서는 레고 부스트에서 어떤 역할을 하나요?,[부터 어렵게 여러 가지가 있는 게 아니라 기본적인 룰이 있고 단계를 넘어가면서 배...,"색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면 기분 좋은 소...",simple,[{}],True
6,초원 산책의 의미와 화자의 감정 변화는 어떤 관계인가요?,[대기 여물지 않은 몇 송이만 남겨놓곤 며칠간의 새벽을 틈타 왕창 털리고 말았다. ...,주어진 질문에 대한 답변이 맥락에 없습니다.,reasoning,[{}],True
7,"output: ""레고 부스트의 색상/거리 센서 역할은?",[부터 어렵게 여러 가지가 있는 게 아니라 기본적인 룰이 있고 단계를 넘어가면서 배...,"레고 부스트의 색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면...",reasoning,[{}],True
8,"question: ""캐나다 유학의 장점은?",[캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이...,주어진 질문에 대한 답변이 맥락에 없습니다.,reasoning,[{}],True
9,"output: ""DK 잡페어 프로그램 내용과 예비 디자이너 기회는?",[간을 가졌다. 뿐만 아니라 DK 토크관에서는 이슈를 만드는 사람과의 만남 디자인 ...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,multi_context,[{}],True


In [ ]:
testset

TestDataset(test_data=[DataRow(question='캐나다에서 대학을 선택할 때 고려해야 할 주요 요소는 무엇인가요?', contexts=[' 우수한 교육 프로그램, 경이로운 자연, 다문화 사회 및 저렴한 생활비를 제공합니다. 유학에 관심이있는 학생들에게 캐나다는 다양한 학업 프로그램을 제공하는 독특한 교육 시스템을 갖춘 안전하고 개방적이며 관용적 인 다문화 사회입니다. 입학을 계획하기 전에 미국과 캐나다 대학에 대해 알아보십시오. 링크를 클릭하고 귀하의 요구 사항에 가장 적합한 국가를 찾으십시오. 캐나다에서 가장 아름다운 10 대 대학을 찾는 것은 매우 쉽고 어려웠습니다. 첫째, 캐나다에는 약 96 개의 대학이 있고 너무 많은 대학이 놀랍게 보입니다. 둘째, 여러 소스가 이전에 동일한 목록을 만들려고 시도했기 때문에 모두 다른 결과를 얻었습니다. 그래도 말이됩니다. 매우 주관적입니다. 대학 외에 공부할 곳을 선택할 때 가장 두려운 부분 중 하나는 살 곳을 결정하는 것입니다. 캠퍼스에 머물러야합니까? 집을 빌려야하나요? 너무 비싸면? 학교에 일종의 Facebook 그룹이있는 경우 일부 사람들이 함께 이사 할 계획을 세울 �', '��도 있습니다. 대학에서 기숙사에 머물기를 기대합니까? 솔직히 어느 쪽도 다른 쪽보다 낫습니다. 캠퍼스 내 또는 캠퍼스 밖에는 많은 변수가 있지만 좀 더 구체적으로 살펴 보겠습니다. 정말, 당신의 선택은 무엇입니까? 캐나다는 세계 최고의 교육 시스템 중 하나를 보유하고 있으며 마스터 포털 은 7 위에 올랐을뿐만 아니라 캐나다는 가장 교육을 많이받은 국가 중 하나입니다. 이는 부분적으로 캐나다 정부가 교육에 투입 한 노력 때문일 수 있습니다. 공부할 곳을 결정할 때 대학의 국제적인 측면은 어떤 식 으로든 요소입니다. 우리가 모국에서 온 다른 학생들이 있을지 알고 싶거나 우리가 찾을 수있는 커뮤니티의 종류를 알고 싶기 때문일 수 있습니다. 그럼에도 불구하고“국제”교육의 혜택은 끝이 없습니다. 거의 1 년 전인 201

In [ ]:
test_df = testset.to_pandas()

In [ ]:
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,노량해전의 결과와 일본 해군의 변화에 대해 설명해 주세요.,[��상자 10명을 제외하고는 피해를 본 것이 없었다. 그러나 일본은 전선 47척이...,"노량해전의 결과로 일본은 전선 47척이 격침되고 12척이 나포되었으며 9,000명이...",simple,[{}],True
1,"DK 잡페어에서 어떤 프로그램이 진행되었고, 어떤 기회가 제공되었나요?",[간을 가졌다. 뿐만 아니라 DK 토크관에서는 이슈를 만드는 사람과의 만남 디자인 ...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,simple,[{}],True
2,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점은 무엇인가요?,[대한민국 근대 도시사는 신도시 역사라 불리울 정도로 우리는 다양한 도시건설 노하우...,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점으로는 베트남에서의 신도시 사업...,simple,[{}],True
3,레고 부스트와 함께 아이들의 상상력과 창의력을 어떻게 키울 수 있나요?,"[�� 나오게 할 수도 있는데, 음성을 변조할 수도 있으니 아이들이 너무 좋아 하더...","레고 부스트와 함께 아이들은 직접 움직임을 줄 수 있어 성취감을 느끼고, 교감 능력...",simple,[{}],True
4,기업 승계가 효율적으로 진행된 사례는 어떤 상황에서 발생했나요?,[가가 부도 위기에 처해 IMF로부터 자금을 지원받아 국가 부도를 면한 외환위기 사...,기업 승계가 효율적으로 진행된 사례는 2008년 국제적인 금융위기와 같은 경제 위기...,simple,[{}],True
5,색상/거리 센서는 레고 부스트에서 어떤 역할을 하나요?,[부터 어렵게 여러 가지가 있는 게 아니라 기본적인 룰이 있고 단계를 넘어가면서 배...,"색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면 기분 좋은 소...",simple,[{}],True
6,초원 산책의 의미와 화자의 감정 변화는 어떤 관계인가요?,[대기 여물지 않은 몇 송이만 남겨놓곤 며칠간의 새벽을 틈타 왕창 털리고 말았다. ...,주어진 질문에 대한 답변이 맥락에 없습니다.,reasoning,[{}],True
7,"output: ""레고 부스트의 색상/거리 센서 역할은?",[부터 어렵게 여러 가지가 있는 게 아니라 기본적인 룰이 있고 단계를 넘어가면서 배...,"레고 부스트의 색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면...",reasoning,[{}],True
8,"question: ""캐나다 유학의 장점은?",[캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이...,주어진 질문에 대한 답변이 맥락에 없습니다.,reasoning,[{}],True
9,"output: ""DK 잡페어 프로그램 내용과 예비 디자이너 기회는?",[간을 가졌다. 뿐만 아니라 DK 토크관에서는 이슈를 만드는 사람과의 만남 디자인 ...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,multi_context,[{}],True


# Evaluation

In [ ]:

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.settings import Settings
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(model='gpt-4o-mini')

vector_index = VectorStoreIndex.from_documents(docs)

#Naive-RAG 엔진 정의
query_engine = vector_index.as_query_engine()


In [ ]:
#평가할 항목 import
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    answer_similarity,
    context_recall,
)

metrics = [
    context_precision,
    context_recall,
    faithfulness,
    answer_similarity,
    answer_relevancy
]


In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

#평가용 모델
evaluator_llm = OpenAI(model="gpt-4o-mini",timeout=120)

In [ ]:
# convert to HF dataset
ds = testset.to_dataset()

In [ ]:
testset.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,노량해전의 결과와 일본 해군의 변화에 대해 설명해 주세요.,[��상자 10명을 제외하고는 피해를 본 것이 없었다. 그러나 일본은 전선 47척이...,"노량해전의 결과로 일본은 전선 47척이 격침되고 12척이 나포되었으며 9,000명이...",simple,[{}],True
1,"DK 잡페어에서 어떤 프로그램이 진행되었고, 어떤 기회가 제공되었나요?",[간을 가졌다. 뿐만 아니라 DK 토크관에서는 이슈를 만드는 사람과의 만남 디자인 ...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,simple,[{}],True
2,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점은 무엇인가요?,[대한민국 근대 도시사는 신도시 역사라 불리울 정도로 우리는 다양한 도시건설 노하우...,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점으로는 베트남에서의 신도시 사업...,simple,[{}],True
3,레고 부스트와 함께 아이들의 상상력과 창의력을 어떻게 키울 수 있나요?,"[�� 나오게 할 수도 있는데, 음성을 변조할 수도 있으니 아이들이 너무 좋아 하더...","레고 부스트와 함께 아이들은 직접 움직임을 줄 수 있어 성취감을 느끼고, 교감 능력...",simple,[{}],True
4,기업 승계가 효율적으로 진행된 사례는 어떤 상황에서 발생했나요?,[가가 부도 위기에 처해 IMF로부터 자금을 지원받아 국가 부도를 면한 외환위기 사...,기업 승계가 효율적으로 진행된 사례는 2008년 국제적인 금융위기와 같은 경제 위기...,simple,[{}],True
5,색상/거리 센서는 레고 부스트에서 어떤 역할을 하나요?,[부터 어렵게 여러 가지가 있는 게 아니라 기본적인 룰이 있고 단계를 넘어가면서 배...,"색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면 기분 좋은 소...",simple,[{}],True
6,초원 산책의 의미와 화자의 감정 변화는 어떤 관계인가요?,[대기 여물지 않은 몇 송이만 남겨놓곤 며칠간의 새벽을 틈타 왕창 털리고 말았다. ...,주어진 질문에 대한 답변이 맥락에 없습니다.,reasoning,[{}],True
7,"output: ""레고 부스트의 색상/거리 센서 역할은?",[부터 어렵게 여러 가지가 있는 게 아니라 기본적인 룰이 있고 단계를 넘어가면서 배...,"레고 부스트의 색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면...",reasoning,[{}],True
8,"question: ""캐나다 유학의 장점은?",[캐나다는 3 년 연속 지구상에서 가장 주목할만한 국가로 선포되었습니다. 일반 타이...,주어진 질문에 대한 답변이 맥락에 없습니다.,reasoning,[{}],True
9,"output: ""DK 잡페어 프로그램 내용과 예비 디자이너 기회는?",[간을 가졌다. 뿐만 아니라 DK 토크관에서는 이슈를 만드는 사람과의 만남 디자인 ...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,multi_context,[{}],True


In [ ]:
# 답변이 안되는 QA페어는 제외
exclude_idx = [0,6,8,13,14]

ds_ex = ds.select(
    (
        i for i in range(len(ds))
        if i not in set(exclude_idx)
    )
)

ds_dict = ds_ex.to_dict()

In [ ]:
ds_ex.to_pandas()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,"DK 잡페어에서 어떤 프로그램이 진행되었고, 어떤 기회가 제공되었나요?",[간을 가졌다. 뿐만 아니라 DK 토크관에서는 이슈를 만드는 사람과의 만남 디자인 ...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,simple,[{}],True
1,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점은 무엇인가요?,[대한민국 근대 도시사는 신도시 역사라 불리울 정도로 우리는 다양한 도시건설 노하우...,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점으로는 베트남에서의 신도시 사업...,simple,[{}],True
2,레고 부스트와 함께 아이들의 상상력과 창의력을 어떻게 키울 수 있나요?,"[�� 나오게 할 수도 있는데, 음성을 변조할 수도 있으니 아이들이 너무 좋아 하더...","레고 부스트와 함께 아이들은 직접 움직임을 줄 수 있어 성취감을 느끼고, 교감 능력...",simple,[{}],True
3,기업 승계가 효율적으로 진행된 사례는 어떤 상황에서 발생했나요?,[가가 부도 위기에 처해 IMF로부터 자금을 지원받아 국가 부도를 면한 외환위기 사...,기업 승계가 효율적으로 진행된 사례는 2008년 국제적인 금융위기와 같은 경제 위기...,simple,[{}],True
4,색상/거리 센서는 레고 부스트에서 어떤 역할을 하나요?,[부터 어렵게 여러 가지가 있는 게 아니라 기본적인 룰이 있고 단계를 넘어가면서 배...,"색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면 기분 좋은 소...",simple,[{}],True
5,"output: ""레고 부스트의 색상/거리 센서 역할은?",[부터 어렵게 여러 가지가 있는 게 아니라 기본적인 룰이 있고 단계를 넘어가면서 배...,"레고 부스트의 색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면...",reasoning,[{}],True
6,"output: ""DK 잡페어 프로그램 내용과 예비 디자이너 기회는?",[간을 가졌다. 뿐만 아니라 DK 토크관에서는 이슈를 만드는 사람과의 만남 디자인 ...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,multi_context,[{}],True
7,question: '직급정년 임금삭감제도' 동의 절차와 의견 교환의 실질성은?,"[�� 의견 교환 및 의견 번복 기회를 부여하였음을 인정할 아무런 자료가 없으므로,...",'직급정년 도달자 임금삭감제도'에 대한 동의 절차와 의견 교환의 실질성은 인정되지 ...,multi_context,"[{}, {}]",True
8,수도권 물류 시장의 공실률 변화와 오피스 자산 거래 규모와의 관계는?,"[�, 평균 연면적 8만㎡ 규모의 대형 자산이 주를 이뤘다. 지속되는 대량 신규 공...",수도권 물류 시장의 평균 공실률은 지난해 말 대비 약 0.8% 포인트 하락하여 2....,multi_context,"[{}, {}]",True
9,광주시는 시민 참여 예산으로 어떤 정책을 제안해야 한다고 보나요?,[�의 투명성을 어떻게 확보할 것인가? 셋째는 광주정보문화산업진흥원을 중심으로 한 ...,광주시는 시민 중심의 '시정 참여와 행정권 감시'를 위한 보다 적극적인 커뮤니케이션...,conditional,[{}],True


In [ ]:
from ragas.integrations.llama_index import evaluate

# Naive-RAG Pipeline으로 테스트셋 질문에 대한 답변 생성 및 사전 import한 평가지표 산출
result = evaluate(
    query_engine=query_engine,
    metrics=metrics,
    dataset=ds_dict,
    llm=evaluator_llm,
    embeddings=embeddings,
    raise_exceptions=False
)


Running Query Engine:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

# Faithfulness: Retrieved Context 를 얼마나 잘 참조한 답변인지
# answer_relevancy: Q&A 연관성. 그냥 얼마나 잘 답변했는지
# context_precision: GT의 내용과 연관된 Context간의 순서가 Relevancy 순으로 정렬되어있는지
# context_recall: GT의 내용과 얼마나 context가 relevant 한지
# Answer semantic similarity: Answer vs GT


In [ ]:
# 종합 평가점수 확인
result

{'context_precision': 1.0000, 'context_recall': 0.7667, 'faithfulness': 0.6775, 'answer_similarity': 0.7815, 'answer_relevancy': 0.4562}

In [ ]:
# 세부항목 확인
result.to_pandas()

,question,contexts,answer,ground_truth,context_precision,context_recall,faithfulness,answer_similarity,answer_relevancy
0,"DK 잡페어에서 어떤 프로그램이 진행되었고, 어떤 기회가 제공되었나요?",[실제로 전시장은 메인 동선에 따른 DX 주제 전시가 진행되고 있었다. 행사장에 들...,"DK 잡페어에서는 디자인 관련 다양한 프로그램이 진행되었으며, 참가자들에게 디자인 ...",DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,1.0,0.000000,1.000000,0.742778,0.641065
1,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점은 무엇인가요?,[대한민국 근대 도시사는 신도시 역사라 불리울 정도로 우리는 다양한 도시건설 노하우...,한국 건설사가 해외 신도시 사업에서 겪고 있는 문제점은 주로 로컬 파트너와의 긴밀한...,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점으로는 베트남에서의 신도시 사업...,1.0,1.000000,0.714286,0.618967,0.661172
2,레고 부스트와 함께 아이들의 상상력과 창의력을 어떻게 키울 수 있나요?,[레고 부스트를 만난 지 2달 정도 되었는데 생각보다 아이들이 잘 따라 만들고 너무...,"레고 부스트를 통해 아이들은 다양한 모델을 만들고, 직접 움직임을 주면서 성취감을 ...","레고 부스트와 함께 아이들은 직접 움직임을 줄 수 있어 성취감을 느끼고, 교감 능력...",1.0,1.000000,0.750000,0.736378,0.362553
3,기업 승계가 효율적으로 진행된 사례는 어떤 상황에서 발생했나요?,[이상에서 살펴본 바와 같이 상장주식을 증여할 때의 평가는 증여일 이전‧이후 각 2...,기업 승계가 효율적으로 진행된 사례는 경제환경이 급변하여 주가가 폭등하거나 폭락하는...,기업 승계가 효율적으로 진행된 사례는 2008년 국제적인 금융위기와 같은 경제 위기...,1.0,1.000000,1.000000,0.769121,0.264661
4,색상/거리 센서는 레고 부스트에서 어떤 역할을 하나요?,[얼마 전 소개해 드린 레고 부스트 17101 기억하시나요? 레고를 통해 만드는 재...,색상/거리 센서는 레고 부스트에서 자신이 해야 할 일을 판단하는 역할을 합니다. 이...,"색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면 기분 좋은 소...",1.0,0.500000,0.333333,0.744763,0.320631
5,"output: ""레고 부스트의 색상/거리 센서 역할은?",[얼마 전 소개해 드린 레고 부스트 17101 기억하시나요? 레고를 통해 만드는 재...,레고 부스트의 색상/거리 센서는 주변 환경을 인식하여 자신이 해야 할 일을 판단하는...,"레고 부스트의 색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면...",1.0,0.500000,0.000000,0.742134,0.835569
6,"output: ""DK 잡페어 프로그램 내용과 예비 디자이너 기회는?",[실제로 전시장은 메인 동선에 따른 DX 주제 전시가 진행되고 있었다. 행사장에 들...,DK 잡페어에서는 국내·외 기업 50여 개가 참석하여 600여 명의 취업 관련 상담...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,1.0,1.000000,NaN,0.944527,0.566363
7,question: '직급정년 임금삭감제도' 동의 절차와 의견 교환의 실질성은?,"[대기발령 만료후 보직 미부여자 인사사항 개선, Ⅷ. 여성근로자를 위한 ‘모성보호’...",'직급정년 임금삭감제도'에 대한 동의 절차와 의견 교환의 실질성은 부족하다고 평가된...,'직급정년 도달자 임금삭감제도'에 대한 동의 절차와 의견 교환의 실질성은 인정되지 ...,1.0,1.000000,0.800000,0.779178,0.000000
8,수도권 물류 시장의 공실률 변화와 오피스 자산 거래 규모와의 관계는?,[물류 시장의 경우 수도권 A급 물류 시장의 평균 임대료가 지난 하반기 대비 약 0...,수도권 물류 시장의 공실률은 지난해 말 대비 약 0.8% 포인트 하락하여 2.4% ...,수도권 물류 시장의 평균 공실률은 지난해 말 대비 약 0.8% 포인트 하락하여 2....,1.0,0.666667,0.500000,0.933395,0.350553
9,광주시는 시민 참여 예산으로 어떤 정책을 제안해야 한다고 보나요?,[민선6기 광주시장에 시민활동가 출신이 당선된 이후 시민과 소통하고 시민의 의견을 ...,광주시는 시민 참여 예산을 통해 시민의 지방자치 참여를 증진시키기 위한 정책을 제안...,광주시는 시민 중심의 '시정 참여와 행정권 감시'를 위한 보다 적극적인 커뮤니케이션...,1.0,1.000000,1.000000,0.803632,0.559425


In [ ]:


print(result.to_pandas().iloc[5]['contexts'])


['얼마 전 소개해 드린 레고 부스트 17101 기억하시나요? 레고를 통해 만드는 재미는 물론 코딩을 통해 직접 움직이고 다양한 게임도 즐기고 즐거움을 주는 코딩 브릭인데요. 아이들에게 쉽고 재밌게 코딩 교육을 해 줄 수 있기 때문에 많은 사람들이 선물로 혹은 코딩 교육의 목적으로 선택하고 있습니다. 지난번에는 움직이는 로봇 버니를 통해 간단한 레고 부스트 소개와 코딩의 기초를 보여 드렸는데요. 오늘은 귀염둥이 고양이 프랭키를 통해 교감하는 애완동물을 만들어 보도록 하겠습니다. 레고 부스트는 하나로 5가지를 만들 수 있는 5-in-1 제품이라 다양한 방식으로 체험할 수 있기 때문에 단순히 만들고 분해하기를 반복하는 것보다 더 흥미 유발이 되더라구요. 버니를 분해해야 한다고 하니 많이 아쉬워 하긴 하더라구요. 하나를 만들고 나면 다시 분해해서 다른 것을 만들어야 하기 때문에 부품의 분실이나 보관에 신경을 써야 하는데요. 이런저런 방법을 생각하다가 일단 락앤락 보관 통을 이용해 봤습니다. 하나만 있어도 꽉 차게 들어가긴 하는데, 너무 꽉 차면 만들 때 불편하고 완전히 분해하면 한 통으로 모자라기 때문에 2개 정도 준비해 주는 게 좋습니다. 저는 기본적으론 색깔별로 분리하고, 연결 부품 따로 넓은 것 혹은 긴것 이런 식으로 모아 놓으니까 찾기도 편하고 만들기가 쉽더라구요. 레고 부스트는 다른 레고와 다르게 스마트폰이나 태블릿을 이용해 설명서를 보면서 만들도록 되어 있습니다. 나중에 다 만들고 나면 코딩까지 연계하기 위한 방법인데, 자유롭게 확대와 축소가 가능하고 훼손 가능성도 적으니 다른 레고도 이렇게 설명서를 제공해 주면 좋겠다는 생각이 들 정도로 상당히 편리합니다. 물론 스마트폰보다는 화면이 큰 태블릿이 보기 편한데, 안드로이드와 아이폰 모두 지원하고 버전별로 호환 가능 여부가 다르니 아래 링크에서 확인할 수 있습니다. 지난번 움직이는 로봇 버니를 만들어 봐서 그런지 이번에는 별로 물어보지 않고 혼자서 착착 만들어 가더라구요.'
 '거리와 색상을 통해 자신이 해

In [ ]:
print(result.to_pandas().iloc[5]['answer'])

레고 부스트의 색상/거리 센서는 주변 환경을 인식하여 자신이 해야 할 일을 판단하는 역할을 합니다. 이를 통해 다양한 동작을 수행하고 감정을 표현하는 데 활용됩니다.


# Rerank 추가한 RAG 비교

In [ ]:
!pip install llama-index-postprocessor-cohere-rerank

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.5 MB/s eta 0:00:00


In [ ]:
import os
from llama_index.postprocessor.cohere_rerank import CohereRerank
#pip install llama-index-postprocessor-cohere-rerank

#코히어 리랭크 api 설정
api_key = ''
cohere_rerank = CohereRerank(api_key=api_key, top_n=2)

In [ ]:
# 리랭크가 반영된 Advanced Pipeline 정의
query_engine_cohere = vector_index.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[cohere_rerank],
)

In [ ]:
result_rerank = evaluate(
    query_engine=query_engine_cohere,
    metrics=metrics,
    dataset=ds_dict,
    llm=evaluator_llm,
    embeddings=embeddings,
    raise_exceptions=False
)


Running Query Engine:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
result

{'context_precision': 1.0000, 'context_recall': 0.7667, 'faithfulness': 0.6775, 'answer_similarity': 0.7815, 'answer_relevancy': 0.4562}

In [ ]:
result_rerank

{'context_precision': 1.0000, 'context_recall': 0.9000, 'faithfulness': 0.6794, 'answer_similarity': 0.7900, 'answer_relevancy': 0.4567}

In [ ]:
result_rerank.to_pandas()

,question,contexts,answer,ground_truth,context_precision,context_recall,faithfulness,answer_similarity,answer_relevancy
0,"DK 잡페어에서 어떤 프로그램이 진행되었고, 어떤 기회가 제공되었나요?","[강금석 카이스트 교수, 박지은 펄스나인 대표, 조중현 그래픽 디자이너, 전은경 월...",DK 잡페어에서는 국내·외 기업 50여 개가 참석하여 600여 명의 취업 관련 상담...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,1.0,1.0,1.000000,0.958809,0.543600
1,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점은 무엇인가요?,[대한민국 근대 도시사는 신도시 역사라 불리울 정도로 우리는 다양한 도시건설 노하우...,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점은 주로 두 가지로 요약될 수 ...,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점으로는 베트남에서의 신도시 사업...,1.0,1.0,1.000000,0.613437,0.661172
2,레고 부스트와 함께 아이들의 상상력과 창의력을 어떻게 키울 수 있나요?,[레고 부스트를 만난 지 2달 정도 되었는데 생각보다 아이들이 잘 따라 만들고 너무...,레고 부스트를 통해 아이들은 직접 움직임을 주고 다양한 감정을 표현하는 고양이 프랭...,"레고 부스트와 함께 아이들은 직접 움직임을 줄 수 있어 성취감을 느끼고, 교감 능력...",1.0,1.0,0.800000,0.781127,0.864659
3,기업 승계가 효율적으로 진행된 사례는 어떤 상황에서 발생했나요?,[최근 언론보도에 의하면 이재현 CJ그룹 회장이 지난해 말 두 자녀에게 증여하였던 ...,"기업 승계가 효율적으로 진행된 사례는 경제환경이 급변하는 상황, 특히 주가가 폭락하...",기업 승계가 효율적으로 진행된 사례는 2008년 국제적인 금융위기와 같은 경제 위기...,1.0,1.0,1.000000,0.752961,0.384633
4,색상/거리 센서는 레고 부스트에서 어떤 역할을 하나요?,"[물론 스마트폰보다는 화면이 큰 태블릿이 보기 편한데, 안드로이드와 아이폰 모두 지...",색상/거리 센서는 레고 부스트에서 물체를 감지하는 역할을 합니다. 이 센서를 통해 ...,"색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면 기분 좋은 소...",1.0,1.0,0.000000,0.945562,0.317749
5,"output: ""레고 부스트의 색상/거리 센서 역할은?","[물론 스마트폰보다는 화면이 큰 태블릿이 보기 편한데, 안드로이드와 아이폰 모두 지...","색상/거리 센서는 물체를 감지하는 역할을 하며, 이를 통해 자신이 해야 할 일을 판...","레고 부스트의 색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면...",1.0,1.0,0.000000,0.621288,0.353640
6,"output: ""DK 잡페어 프로그램 내용과 예비 디자이너 기회는?","[강금석 카이스트 교수, 박지은 펄스나인 대표, 조중현 그래픽 디자이너, 전은경 월...",DK 잡페어에서는 국내·외 기업 50여 개가 참석하여 600여 명의 취업 관련 상담...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,1.0,1.0,0.875000,0.933548,0.480136
7,question: '직급정년 임금삭감제도' 동의 절차와 의견 교환의 실질성은?,"[5. 22. 금요일 18:00를 기준으로 위 문서를 열람한 사람 수는 총 1,50...",'직급정년 임금삭감제도'에 대한 동의 절차와 의견 교환의 실질성은 부족한 것으로 평...,'직급정년 도달자 임금삭감제도'에 대한 동의 절차와 의견 교환의 실질성은 인정되지 ...,1.0,1.0,1.000000,0.749081,0.000000
8,수도권 물류 시장의 공실률 변화와 오피스 자산 거래 규모와의 관계는?,[물류 시장의 경우 수도권 A급 물류 시장의 평균 임대료가 지난 하반기 대비 약 0...,수도권 물류 시장의 공실률 변화는 오피스 자산 거래 규모와 밀접한 관계가 있다. 물...,수도권 물류 시장의 평균 공실률은 지난해 말 대비 약 0.8% 포인트 하락하여 2....,1.0,0.0,0.285714,0.747458,0.406930
9,광주시는 시민 참여 예산으로 어떤 정책을 제안해야 한다고 보나요?,[민선6기 광주시장에 시민활동가 출신이 당선된 이후 시민과 소통하고 시민의 의견을 ...,광주시는 시민 참여 예산을 통해 시민의 지방자치 참여를 증진시키기 위한 정책을 제안...,광주시는 시민 중심의 '시정 참여와 행정권 감시'를 위한 보다 적극적인 커뮤니케이션...,1.0,1.0,0.833333,0.796550,0.554648


In [ ]:
result.to_pandas()

,question,contexts,answer,ground_truth,context_precision,context_recall,faithfulness,answer_similarity,answer_relevancy
0,"DK 잡페어에서 어떤 프로그램이 진행되었고, 어떤 기회가 제공되었나요?",[실제로 전시장은 메인 동선에 따른 DX 주제 전시가 진행되고 있었다. 행사장에 들...,"DK 잡페어에서는 디자인 관련 다양한 프로그램이 진행되었으며, 참가자들에게 디자인 ...",DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,1.0,0.000000,1.000000,0.742778,0.641065
1,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점은 무엇인가요?,[대한민국 근대 도시사는 신도시 역사라 불리울 정도로 우리는 다양한 도시건설 노하우...,한국 건설사가 해외 신도시 사업에서 겪고 있는 문제점은 주로 로컬 파트너와의 긴밀한...,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점으로는 베트남에서의 신도시 사업...,1.0,1.000000,0.714286,0.618967,0.661172
2,레고 부스트와 함께 아이들의 상상력과 창의력을 어떻게 키울 수 있나요?,[레고 부스트를 만난 지 2달 정도 되었는데 생각보다 아이들이 잘 따라 만들고 너무...,"레고 부스트를 통해 아이들은 다양한 모델을 만들고, 직접 움직임을 주면서 성취감을 ...","레고 부스트와 함께 아이들은 직접 움직임을 줄 수 있어 성취감을 느끼고, 교감 능력...",1.0,1.000000,0.750000,0.736378,0.362553
3,기업 승계가 효율적으로 진행된 사례는 어떤 상황에서 발생했나요?,[이상에서 살펴본 바와 같이 상장주식을 증여할 때의 평가는 증여일 이전‧이후 각 2...,기업 승계가 효율적으로 진행된 사례는 경제환경이 급변하여 주가가 폭등하거나 폭락하는...,기업 승계가 효율적으로 진행된 사례는 2008년 국제적인 금융위기와 같은 경제 위기...,1.0,1.000000,1.000000,0.769121,0.264661
4,색상/거리 센서는 레고 부스트에서 어떤 역할을 하나요?,[얼마 전 소개해 드린 레고 부스트 17101 기억하시나요? 레고를 통해 만드는 재...,색상/거리 센서는 레고 부스트에서 자신이 해야 할 일을 판단하는 역할을 합니다. 이...,"색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면 기분 좋은 소...",1.0,0.500000,0.333333,0.744763,0.320631
5,"output: ""레고 부스트의 색상/거리 센서 역할은?",[얼마 전 소개해 드린 레고 부스트 17101 기억하시나요? 레고를 통해 만드는 재...,레고 부스트의 색상/거리 센서는 주변 환경을 인식하여 자신이 해야 할 일을 판단하는...,"레고 부스트의 색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면...",1.0,0.500000,0.000000,0.742134,0.835569
6,"output: ""DK 잡페어 프로그램 내용과 예비 디자이너 기회는?",[실제로 전시장은 메인 동선에 따른 DX 주제 전시가 진행되고 있었다. 행사장에 들...,DK 잡페어에서는 국내·외 기업 50여 개가 참석하여 600여 명의 취업 관련 상담...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,1.0,1.000000,NaN,0.944527,0.566363
7,question: '직급정년 임금삭감제도' 동의 절차와 의견 교환의 실질성은?,"[대기발령 만료후 보직 미부여자 인사사항 개선, Ⅷ. 여성근로자를 위한 ‘모성보호’...",'직급정년 임금삭감제도'에 대한 동의 절차와 의견 교환의 실질성은 부족하다고 평가된...,'직급정년 도달자 임금삭감제도'에 대한 동의 절차와 의견 교환의 실질성은 인정되지 ...,1.0,1.000000,0.800000,0.779178,0.000000
8,수도권 물류 시장의 공실률 변화와 오피스 자산 거래 규모와의 관계는?,[물류 시장의 경우 수도권 A급 물류 시장의 평균 임대료가 지난 하반기 대비 약 0...,수도권 물류 시장의 공실률은 지난해 말 대비 약 0.8% 포인트 하락하여 2.4% ...,수도권 물류 시장의 평균 공실률은 지난해 말 대비 약 0.8% 포인트 하락하여 2....,1.0,0.666667,0.500000,0.933395,0.350553
9,광주시는 시민 참여 예산으로 어떤 정책을 제안해야 한다고 보나요?,[민선6기 광주시장에 시민활동가 출신이 당선된 이후 시민과 소통하고 시민의 의견을 ...,광주시는 시민 참여 예산을 통해 시민의 지방자치 참여를 증진시키기 위한 정책을 제안...,광주시는 시민 중심의 '시정 참여와 행정권 감시'를 위한 보다 적극적인 커뮤니케이션...,1.0,1.000000,1.000000,0.803632,0.559425


In [ ]:


print(result_rerank.to_pandas().iloc[0]['contexts'])


['강금석 카이스트 교수, 박지은 펄스나인 대표, 조중현 그래픽 디자이너, 전은경 월간 <디자인> 편집장 등 디자인과 기술, 크리에이티브 등 여러 분야의 연사들이 참석해 디자인과 관련된 다양한 이야기를 들려줬다. 이 밖에도 전시장에는 디자인을 위한 여러 공간이 마련돼 있었다. 예비 디자이너와 청년 디자이너들의 취·창업을 돕기 위한 프로그램이 곳곳에서 열렸는데, DK 상담장과 DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과 구인 면접을 진행하기도 했다. 또한 DK 잡페어에는 실제 청년 디자이너 120명의 작품을 전시해 디자인 관계자들에게 자신의 작품을 알리는 디자이너 쇼케이스가 마련됐다. 뿐만 아니라 디자인 문화 콘텐츠 구직자 취업 상담회와 비즈니스 상담회, 해외 취·창업 그룹 멘토링이 진행돼 예비 디자이너들의 큰 호응을 얻었다. 한편, DK 어워드관에는 한국청소년디자인전람회 수상작과 대한민국전람회 수상작, 글로벌 생활명품 선정작 등의 전시가 진행되기도 했다. 이로써 이번 디자인코리아 페스티벌 2019는 국내의 우수한 디자인은 물론, 해외의 다양한 디자인을 함께 접하는 자리가 됐다. 또한 전시와 포럼, 강연 등의 유익한 행사가 진행돼 디자인에 대해 심도 있게 고민해보고 이야기를 나눌 수 있었던 의미 있는 행사였다. 뉴스콘텐츠는 저작권법 제7조 규정된 단서조항을 제외한 저작물로서 저작권법의 보호대상입니다. 본 기사를 개인블로그 및 홈페이지, 카페 등에 게재(링크)를 원하시는 분은 반드시 기사의 출처(로고)를 붙여주시기 바랍니다. 영리를 목적으로 하지 않더라도 출처 없이 본 기사를 재편집해 올린 해당 미디어에 대해서는 합법적인 절차(지적재산권법)에 따라 그 책임을 묻게 되며, 이에 따른 불이익은 책임지지 않습니다.'
 '실제로 전시장은 메인 동선에 따른 DX 주제 전시가 진행되고 있었다. 행사장에 들어서면 거대한 미디어로드를 만나볼 수 있는데 이곳에서 방문자들은 디지털 아트와 디지털 애니메이션을 통해 DX의 정의를 이해할 수 있었

In [ ]:
print(result_rerank.to_pandas().iloc[0]['question'])

DK 잡페어에서 어떤 프로그램이 진행되었고, 어떤 기회가 제공되었나요?


In [ ]:
print(result_rerank.to_pandas().iloc[0]['answer'])

DK 잡페어에서는 국내·외 기업 50여 개가 참석하여 600여 명의 취업 관련 상담과 구인 면접이 진행되었습니다. 또한, 실제 청년 디자이너 120명의 작품이 전시되어 디자인 관계자들에게 자신의 작품을 알릴 수 있는 기회가 제공되었습니다. 이 외에도 디자인 문화 콘텐츠 구직자 취업 상담회와 비즈니스 상담회, 해외 취·창업 그룹 멘토링 프로그램이 열려 예비 디자이너들에게 다양한 지원과 기회를 제공했습니다.


In [ ]:
print(result.to_pandas().iloc[0]['answer'])

DK 잡페어에서는 디자인 관련 다양한 프로그램이 진행되었으며, 참가자들에게 디자인 산업의 현황과 미래에 대한 통찰을 제공하는 기회가 마련되었습니다. 이 행사에서는 국내외 디자인 전문가들이 참여하여 디자인과 기술, 크리에이티브 분야에 대한 강연과 토론이 이루어졌습니다. 또한, 참가자들은 디자인 관련 기업과의 네트워킹 기회를 통해 취업 및 경력 개발에 도움이 되는 정보를 얻을 수 있었습니다.


# 오늘 배운 기술 중 택1 해서 평가 해보세요!

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display
import pprint
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(model='gpt-4o-mini')

In [ ]:
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine, hyde)

In [ ]:
result_rerank = evaluate(
    query_engine=hyde_query_engine,
    metrics=metrics,
    dataset=ds_dict,
    llm=evaluator_llm,
    embeddings=embeddings,
    raise_exceptions=False
)


Running Query Engine:   0%|          | 0/10 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
result_rerank

{'context_precision': 1.0000, 'context_recall': 0.8667, 'faithfulness': 0.7260, 'answer_similarity': 0.7987, 'answer_relevancy': 0.4525}

In [ ]:
result

{'context_precision': 1.0000, 'context_recall': 0.7667, 'faithfulness': 0.6775, 'answer_similarity': 0.7815, 'answer_relevancy': 0.4562}

In [ ]:
result_rerank.to_pandas()

,question,contexts,answer,ground_truth,context_precision,context_recall,faithfulness,answer_similarity,answer_relevancy
0,"DK 잡페어에서 어떤 프로그램이 진행되었고, 어떤 기회가 제공되었나요?",[실제로 전시장은 메인 동선에 따른 DX 주제 전시가 진행되고 있었다. 행사장에 들...,"DK 잡페어에서는 다양한 프로그램이 진행되었으며, 참가자들에게 여러 기회가 제공되었...",DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,1.0,0.000000,NaN,0.701653,0.735701
1,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점은 무엇인가요?,[대한민국 근대 도시사는 신도시 역사라 불리울 정도로 우리는 다양한 도시건설 노하우...,한국 건설사가 해외 신도시 사업에서 겪고 있는 문제점은 여러 가지가 있습니다. 첫째...,해외 신도시 사업에서 한국 건설사가 겪고 있는 문제점으로는 베트남에서의 신도시 사업...,1.0,1.000000,1.000000,0.584063,0.602939
2,레고 부스트와 함께 아이들의 상상력과 창의력을 어떻게 키울 수 있나요?,[레고 부스트를 만난 지 2달 정도 되었는데 생각보다 아이들이 잘 따라 만들고 너무...,레고 부스트는 아이들이 직접 움직임을 줄 수 있는 경험을 통해 성취감을 느끼게 하며...,"레고 부스트와 함께 아이들은 직접 움직임을 줄 수 있어 성취감을 느끼고, 교감 능력...",1.0,1.000000,0.692308,0.766086,0.773733
3,기업 승계가 효율적으로 진행된 사례는 어떤 상황에서 발생했나요?,[이상에서 살펴본 바와 같이 상장주식을 증여할 때의 평가는 증여일 이전‧이후 각 2...,기업 승계가 효율적으로 진행된 사례는 경제환경이 급변하여 주가가 폭등하거나 폭락하는...,기업 승계가 효율적으로 진행된 사례는 2008년 국제적인 금융위기와 같은 경제 위기...,1.0,1.000000,0.833333,0.769121,0.264661
4,색상/거리 센서는 레고 부스트에서 어떤 역할을 하나요?,[얼마 전 소개해 드린 레고 부스트 17101 기억하시나요? 레고를 통해 만드는 재...,"색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면 기분 좋은 소...","색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면 기분 좋은 소...",1.0,1.000000,0.500000,0.958226,0.320631
5,"output: ""레고 부스트의 색상/거리 센서 역할은?",[얼마 전 소개해 드린 레고 부스트 17101 기억하시나요? 레고를 통해 만드는 재...,레고 부스트의 색상/거리 센서는 주변 환경을 인식하여 자신이 해야 할 일을 판단하는...,"레고 부스트의 색상/거리 센서는 물체를 감지하는 역할을 하며, 쓰다듬는 동작을 하면...",1.0,1.000000,0.250000,0.728013,0.355482
6,"output: ""DK 잡페어 프로그램 내용과 예비 디자이너 기회는?",[실제로 전시장은 메인 동선에 따른 DX 주제 전시가 진행되고 있었다. 행사장에 들...,DK 잡페어에서는 국내·외 기업 50여 개가 참석하여 600여 명의 취업 관련 상담...,DK 잡페어에서는 국내·외 기업 50여 개가 참석해 600여 명의 취업 관련 상담과...,1.0,1.000000,1.000000,0.944527,0.566363
7,question: '직급정년 임금삭감제도' 동의 절차와 의견 교환의 실질성은?,"[대기발령 만료후 보직 미부여자 인사사항 개선, Ⅷ. 여성근로자를 위한 ‘모성보호’...",'직급정년 임금삭감제도'에 대한 동의 절차와 의견 교환의 실질성은 부족하다고 평가된...,'직급정년 도달자 임금삭감제도'에 대한 동의 절차와 의견 교환의 실질성은 인정되지 ...,1.0,1.000000,0.833333,0.798522,0.000000
8,수도권 물류 시장의 공실률 변화와 오피스 자산 거래 규모와의 관계는?,[물류 시장의 경우 수도권 A급 물류 시장의 평균 임대료가 지난 하반기 대비 약 0...,수도권 물류 시장의 공실률은 지난해 말 대비 약 0.8% 포인트 하락하여 2.4% ...,수도권 물류 시장의 평균 공실률은 지난해 말 대비 약 0.8% 포인트 하락하여 2....,1.0,0.666667,0.625000,0.943255,0.351106
9,광주시는 시민 참여 예산으로 어떤 정책을 제안해야 한다고 보나요?,[민선6기 광주시장에 시민활동가 출신이 당선된 이후 시민과 소통하고 시민의 의견을 ...,광주시는 시민 참여 예산을 통해 시민의 지방자치 참여를 증진시키는 정책을 제안해야 ...,광주시는 시민 중심의 '시정 참여와 행정권 감시'를 위한 보다 적극적인 커뮤니케이션...,1.0,1.000000,0.800000,0.793654,0.554648
